In [ ]:
from polymorph_s2df import *
import jax.numpy as jnp
import jax

SAMPLE_SIZE = 10000
INTEGRATION_BOUNDS = (-3, 3)
SEED = jax.random.PRNGKey(0)
points = jax.random.uniform(SEED, (SAMPLE_SIZE, 2), jnp.float32, *INTEGRATION_BOUNDS)

In [ ]:
################################
# Direct Optimistix optimization
# (not using our stuff)

def area(shape, points, bounds):
    samples = shape.is_inside(points)
    return samples.mean() * (bounds[1] - bounds[0])**2


import optimistix
from timeit import default_timer as timer
def optimize_params(cost, params, points):
    solver = optimistix.BFGS(rtol=1e-5, atol=1e-6)
    start = timer()
    solution = optimistix.minimise(cost, solver, params, points, throw=False)
    elapsed = timer() - start
    print("{0} steps in {1:.2f} seconds".format(
            solution.stats.get('num_steps'),
            elapsed))
    return solution.value

In [ ]:
import polymorph_num.expr as expr
from polymorph_num.expr import as_expr
from polymorph_num import loss, ops, optimizer, vec

apoints = vec.Vec2(as_expr(points[:, 0]), as_expr(points[:, 1]))

def area(f, points, bounds):
    distance = circle_sdf(r, c, points)
    scale = as_expr(100)
    is_inside = as_expr(1) - (scale*distance).sigmoid()
    mean = expr.Sum(is_inside) / is_inside.dim
    return mean * (bounds[1] - bounds[0])**2


In [ ]:
#####################################
# Find radius for target circle area

r = ops.param()
c = vec.Vec2(0.,0.)

def circle_sdf(radius, center, point):
    dx = center.x - point.x
    dy = center.y - point.y
    dist = (dx*dx + dy*dy).sqrt()
    return dist - radius


target_area = as_expr(3.141)
error = target_area - area(circle_sdf, apoints, INTEGRATION_BOUNDS)

l = loss.Loss(error*error)
l.register_output(r)

opt = optimizer.Optimizer(l)
print("---")

def solve():
    start = timer()
    soln = opt.optimize({})
    elapsed = timer() - start
    print("optimized in {0:.2f} seconds".format(elapsed))
    return soln

soln = solve()
soln = solve()

print(soln.eval(r))